# New Customer List Data Cleaning and Analysis

# 1.0 Libraries

■ Pandas for data manipulation

■ NumPy for numerical operations

■ datetime for date-related operations

In [1]:
import math
import pandas as pd
import numpy as np
from datetime import datetime, date

# 2.0 Load Data
Loads customer list data from the Excel file 'Raw_data.xlsx', sheet 'NewCustomerList'

In [2]:
def load_data(file_path, sheet_name):
    """Load data from an Excel file."""
    return pd.read_excel(file_path, sheet_name=sheet_name)

In [3]:
file_path = 'Raw_data.xlsx'
sheet_name = 'NewCustomerList'
new_cust = load_data(file_path, sheet_name)

# 3.0 Drop Irrelevant Columns

In [4]:
def drop_irrelevant_columns(dataframe, columns_to_drop):
    """Drop irrelevant columns from the dataset."""
    dataframe.drop(labels=columns_to_drop, axis=1, inplace=True)

In [5]:
irrelevant_columns = ['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20']
value_mapping_gender = {'M': 'Male', 'F': 'Female'}
drop_irrelevant_columns(new_cust, irrelevant_columns)

# 4.0 Display Data Information

Displays basic information about the dataset, including:

■ Number of rows

■ Column names and data types

■ Number of missing values in each column

■ Summary statistics for numerical columns

In [6]:
def display_data_info(dataframe):
    """Basic information about the dataset."""
    print("Initial data overview:")
    print(dataframe.head(5))
    print(dataframe.info())

In [7]:
# Display basic information
display_data_info(new_cust)

Initial data overview:
  first_name  last_name  gender  past_3_years_bike_related_purchases  \
0    Chickie    Brister    Male                                   86   
1      Morly     Genery    Male                                   69   
2    Ardelis  Forrester  Female                                   10   
3     Lucine      Stutt  Female                                   64   
4    Melinda     Hadlee  Female                                   34   

                   DOB                   job_title job_industry_category  \
0           1957-07-12             General Manager         Manufacturing   
1           1970-03-22         Structural Engineer              Property   
2  1974-08-28 00:00:00      Senior Cost Accountant    Financial Services   
3           1979-01-28  Account Representative III         Manufacturing   
4           1965-09-21           Financial Analyst    Financial Services   

      wealth_segment deceased_indicator owns_car  tenure              address  \
0     

# 5.0 Check Missing Values
checks for missing values in the dataset, providing both the total number and percentage of missing values.

In [8]:
def check_missing_values(dataframe):
    """Check for missing values in the dataset."""
    print("Total number of missing values:")
    print(dataframe.isnull().sum())
    print("\nPercentage of missing values:")
    print(dataframe.isnull().mean() * 100)

In [9]:
# Missing values check
check_missing_values(new_cust)

Total number of missing values:
first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
dtype: int64

Percentage of missing values:
first_name                              0.0
last_name                               2.9
gender                                  0.0
past_3_years_bike_

# 6.0 Fill Missing Last Name

Fills missing last names with 'None'

In [10]:
def fill_missing_last_name(dataframe):
    """Fill missing last names with 'None'."""
    dataframe['last_name'].fillna('None', inplace=True)

In [11]:
fill_missing_last_name(new_cust)

# 7.0 Remove Records with Missing Date of Birth

In [12]:
def remove_records_with_missing_dob(dataframe):
    """Remove records with missing Date of Birth."""
    dataframe.dropna(subset=['DOB'], inplace=True)

In [13]:
remove_records_with_missing_dob(new_cust)

# 8.0 Calculate Age

Creates an 'Age' column based on the Date of Birth.

In [14]:
def calculate_age(dataframe):
    """Create 'Age' column based on the Date of Birth."""
    today = pd.to_datetime(date.today())
    dataframe['DOB'] = pd.to_datetime(dataframe['DOB'], errors='coerce')
    age_in_days = (today - dataframe['DOB']).dt.days
    age_in_years = np.floor_divide(age_in_days, 365)
    dataframe['Age'] = age_in_years

In [15]:
calculate_age(new_cust)

# 9.0 Handle Inconsistencies

Handles inconsistencies in the 'gender' column using value mapping.

In [16]:
def handle_inconsistencies(dataframe, column_name, value_mapping):
    """Handle inconsistencies in a categorical column using value mapping."""
    dataframe[column_name] = dataframe[column_name].map(value_mapping)

In [17]:
#inconsistencies
handle_inconsistencies(new_cust, 'gender', value_mapping_gender)

# 10.0 Create Customer ID

Creates a unique customer ID based on 'first_name' and 'last_name'.

In [18]:
def create_customer_id(row):
    """Create a unique customer ID based on 'first_name' and 'last_name'."""
    return f"{row['first_name']}_{row['last_name']}"

In [19]:
#'customer_id' column creation
new_cust['customer_id'] = new_cust.apply(create_customer_id, axis=1)

In [20]:
# columns to verify the addition of 'customer_id'
print("Columns in DataFrame after adding 'customer_id':", new_cust.columns)

Columns in DataFrame after adding 'customer_id': Index(['first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'address', 'postcode', 'state', 'country',
       'property_valuation', 'Rank', 'Value', 'Age', 'customer_id'],
      dtype='object')


# 11.0 Create Age Group

Creates an 'Age Group' column based on the 'Age' column.

In [21]:
new_cust['Age Group'] = new_cust['Age'].apply(lambda x : (math.floor(x/10)+1)*10)

# 12.0 Drop Duplicates

Removes duplicate records from the dataset based on the 'customer_id'.

In [22]:
def drop_duplicates(dataframe, primary_key_column):
    """Remove duplicate records from the dataset."""
    if primary_key_column in dataframe.columns:
        dataframe_deduped = dataframe.drop_duplicates(subset=[primary_key_column])
        return dataframe_deduped
    else:
        print(f"Column '{primary_key_column}' not found in the DataFrame. Available columns: {dataframe.columns}")
        return dataframe

In [23]:
# Duplicate checks
primary_key_column = 'customer_id'
new_cust_deduped = drop_duplicates(new_cust, primary_key_column)

In [24]:
print("Number of records after removing duplicates: {}".format(new_cust_deduped.shape[0]))

Number of records after removing duplicates: 983


# 13.0 Export Cleaned Data to CSV

Exports the cleaned dataset to a CSV file named 'Cleaned_New_Customer_List_Dataset.csv'.

In [25]:
def export_cleaned_data_to_csv(dataframe, file_name):
    """Export the cleaned dataset to a CSV file."""
    dataframe.to_csv(file_name, index=False)

In [26]:
# Export cleaned data to CSV
export_cleaned_data_to_csv(new_cust_deduped, 'Cleaned_New_Customer_List_Dataset.csv')